In [1]:
import json
import jsonlines
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)



In [2]:
# prompt_rag.update_train_search_llm( 
#     "SOLAR-10.7B-Instruct-v1.0", "solar", port=8001, 
#     data_search=data_search, data_search_llm=data_search_llm) 



合并多个文件的先验知识

In [3]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = 1
num_part = 5




# for i in range(1, num_part + 1):
#     with open(f"{search_engine}/K={K}/train_search_llm_v{i}.json", "r") as f:
#         prior_knowledge_list.extend(json.load(f))

# for i, item in enumerate(data_search_llm):
#     if item["claim"] != prior_knowledge_list[i]["claim"].strip():
#         print(i)
#         print(item["claim"])
#         print(prior_knowledge_list[i]["claim"])
#         raise Exception()
#     else:
#         item[f"prior_knowledge_solar_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i]["prior_knowledge_solar"]

# data_search_llm[0]


In [4]:
# prompt_rag.save_search_llm(data_search_llm, search_engine)


创建数据（带有先验知识的Prompt）以微调LLM

In [5]:
# dict_list = []

# for i, item in enumerate(data_search_llm):
    
#     if item["label"] != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge_v3(
#             item["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_solar_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=item["date"],
#             sort=sort
#         )
#         label = "TRUE." if item["label"] == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[834]["query"])
# len(dict_list), dict_list[:2]


In [6]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# data_version = "1"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_solar_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果

In [7]:
import os
import sys
# import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = ["..", "../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference, get_default_template_type
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType, get_yarn_model_tokenizer

model_type = CustomModelType.solar_instruct_10_7b
template_type = get_default_template_type(model_type)
print(f"Template type: {template_type}")

model, tokenizer = get_model_tokenizer(model_type)
model.generation_config.max_new_tokens = 4096
template = get_template(template_type, tokenizer)


2024-01-25 21:07:11,278 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-25 21:07:11,280 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-25 21:07:11,306 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/SOLAR-10.7B-Instruct-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 48,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_sc

Template type: solar


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i
    
i = 0
# i = get_id("As of 3/7/20, Tanzania and Zambia")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

prompt = prompt_rag.get_prompt_for_generating_prior_knowledge(
    claim, data_search[i]["date"], search_engine, search_results, K=K, sort=False)
print(prompt)

response, history = inference(model, template, prompt)
print(response.strip())


Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

INFORMATION:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe case